In [74]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.linear_model import lasso_path
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
%matplotlib inline

In [101]:
def pad_ones_column(X):
    """Add columns of ones to a data matrix"""
    return np.c_[ np.ones(len(ORIGINAL_DF)), X ]

def estimate_beta(X, y):
    """Compte estimates for beta given X and y
    
    Beta = (X^T X)^-1 X^T y
    """
    t1 = np.linalg.inv(np.matmul(X.T, X))
    t2 = np.matmul(X.T, y)
    return np.matmul(t1, t2)

def run_complete_linear_regression(X, y, beta_hat):
    """Run linear regression and produce error statistics"""
    # Make predictions based on testing data
    predicted_y = np.matmul(X, beta_hat)
    
    # Compute the error from the actual data
    errors = np.square(y - predicted_y)
    
    # Compute mean squared error
    MSE = np.mean(errors)
    
    # Compute sum squared error
    SSE = np.sum(errors)

    return MSE, SSE

def run_k_fold(X, y, n_splits=10, print_stuff=False):
    """Run k-fold cross validation"""
    kf = KFold(n_splits=n_splits)
    storeResultsMSE = []
    storeResultsSSE = []

    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        beta_hat = estimate_beta(X_train, y_train)

        test_MSE, test_SSE = run_complete_linear_regression(X_test, y_test, beta_hat)

        storeResultsMSE.append(test_MSE)
        storeResultsSSE.append(test_SSE)
        
        if print_stuff:
            print("Iteration:", i, "MSE: ", test_MSE, "SSE:", test_SSE)  
    if print_stuff:
        print("Average MSE:", np.mean(storeResultsMSE), "Average SSE:", np.mean(storeResultsSSE))
    
    return np.mean(storeResultsMSE)

In [7]:
data = pd.read_csv("final_data.csv", index_col=0).dropna()

In [128]:
X_d = data.drop(['review/appearance', 'review/aroma', 'review/overall', 'review/palate', 'review/taste', 'avg_palate', 'avg_aroma', 'avg_overall', 'avg_appear', 'avg_taste'], axis=1)
X = np.matrix(X_d)
y = data['review/overall'].values

In [144]:
# Fit Lasso Model for review overall
X = np.matrix(data.drop(['review/appearance', 'review/aroma', 'review/overall', 'review/palate', 'review/taste'], axis=1))
y_overall = data['review/overall'].values
model_overa = LassoCV(cv=10, fit_intercept=True).fit(X,y_overall)

# Fit Lasso Model for review appearance
y_appear = data['review/appearance'].values
model_appear = LassoCV(cv=10, fit_intercept=True).fit(X,y_appear)

# Fit Lasso Model for review aroma
y_aroma = data['review/aroma'].values
model_aroma = LassoCV(cv=10, fit_intercept=True).fit(X,y_aroma)

# Fit Lasso Model for review palate
y_palate = data['review/palate'].values
model_palate = LassoCV(cv=10, fit_intercept=True).fit(X,y_palate)

# Fit Lasso Model for review taste
y_taste = data['review/taste'].values
model_taste = LassoCV(cv=10, fit_intercept=True).fit(X,y_taste)

In [145]:
# Select feature columns for each dataset
ind_overall = [i for i, x in enumerate(model_overa.coef_) if x > 0]
ind_appear = [i for i, x in enumerate(model_appear.coef_) if x > 0]
ind_aroma = [i for i, x in enumerate(model_aroma.coef_) if x > 0]
ind_palate = [i for i, x in enumerate(model_palate.coef_) if x > 0]
ind_taste = [i for i, x in enumerate(model_taste.coef_) if x > 0]

In [146]:
ind_overall == ind_appear == ind_aroma == ind_palate == ind_taste

False

In [147]:
# Select the columns for each variable
X_overall = X[: , ind_overall]
X_appear = X[: , ind_appear]
X_aroma = X[: , ind_aroma]
X_palate = X[: , ind_palate]
X_taste = X[: , ind_taste]

In [148]:
# Overall get MSE
run_k_fold(X_overall, np.matrix(y_overall).T)

0.37931720352214943

In [149]:
# Appearance get MSE
run_k_fold(X_appear, np.matrix(y_appear).T)

0.25193336607437378

In [150]:
# Aroma get MSE
run_k_fold(X_aroma, np.matrix(y_aroma).T)

0.29041628410206644

In [151]:
# Palate get MSE
run_k_fold(X_palate, np.matrix(y_palate).T)

0.30537309387326644

In [152]:
# Taste get MSE
run_k_fold(X_taste, np.matrix(y_taste).T)

0.33206299158102531

In [161]:
# Look at features for Overall
X_d.iloc[:, ind_overall].shape

(37484, 81)

In [162]:
# Features for appear
X_d.iloc[:, ind_appear].shape

(37484, 56)

In [163]:
# Features for aroma
X_d.iloc[:, ind_aroma].shape

(37484, 48)

In [164]:
# Features for palate
X_d.iloc[:, ind_palate].shape

(37484, 51)

In [165]:
# Features for taste
X_d.iloc[:, ind_taste].shape

(37484, 51)

In [168]:
set(X_d.iloc[:, ind_appear].columns).intersection(set(X_d.iloc[:, ind_overall].columns),
                                           set(X_d.iloc[:, ind_aroma].columns),
                                           set(X_d.iloc[:, ind_palate].columns),
                                           set(X_d.iloc[:, ind_taste].columns))

{'0',
 '1.1',
 '1075',
 '1199',
 '14',
 '14879',
 '16386',
 '20.0.1',
 '263',
 '3',
 '394',
 '4',
 '5 Day IPA',
 'Caldera IPA',
 'Founders Breakfast Stout',
 'Founders KBS (Kentucky Breakfast Stout)',
 'Founders Porter',
 "Founders Red's Rye PA",
 'Founders RÃ¼bÃ¦us',
 'Okocim Porter'}